### Import libraries

In [54]:
import numpy as np
import torch
import torch.nn as nn                   # all neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.nn.functional as F         # parametersless functions, like (some) activation functions
import torch.optim as optim             # all optimization algorithms, SGD, Adam, etc
from torch.utils.data import DataLoader # gives easier dataset management and creates mini batches
import torchvision
from torchvision import datasets        # has standard datasets we can import in a nice and easy way
from torchvision import transforms      # transformations we can perform on our dataset (data processing)
import nbimporter
from torchsummary import summary

import matplotlib.pyplot as plt

import preprocessing2

print("Pytorch version:", torch.__version__)

Pytorch version: 1.11.0


In [2]:
x = preprocessing2.get_data("data/sign_mnist_train.csv")

In [5]:
print(type(x))
single_batch = iter(x)
Xs, ys = single_batch.next()
print(Xs.shape)
print(ys.shape)

<class 'torch.utils.data.dataloader.DataLoader'>
torch.Size([64, 1, 28, 28])
torch.Size([64])


### Set device

In [6]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### Hyperparameters

In [32]:
#in_channels = 1    # 28x28 = 784, size of MNIST images (grayscale)
#hidden_size = 100
num_classes = 24
learning_rate = 0.001
batch_size = 64
num_epochs = 2

### Create Network

In [33]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3,3))
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2))

        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3,3))
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2))

        self.conv3 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=(3,3))
        self.dropout1 = nn.Dropout2d()

        self.fc1 = nn.Linear(30*3*3, 270)
        self.fc2 = nn.Linear(270, num_classes)

        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        output = self.conv1(x)
        output = F.relu(output)
        output = self.pool1(output)

        output = self.conv2(output)
        output = F.relu(output)
        output = self.pool2(output)

        output = self.conv3(output)
        output = F.relu(output)
        output = self.dropout1(output)

        output = output.view(-1, 30*3*3)
        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))
        return self.softmax(output)

### 1st Architecture

In [17]:
class ConvNN2(nn.Module):
    def __init__(self):
        super(ConvNN2, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, 3)
        self.pool1 = nn.MaxPool2d(2)
        
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.pool2 = nn.MaxPool2d(2)
        
        self.conv3 = nn.Conv2d(20, 30, 3) 
        self.dropout1 = nn.Dropout2d()
        
        self.fc3 = nn.Linear(30 * 3 * 3, 270) 
        self.fc4 = nn.Linear(270, 26)
        
        self.softmax = nn.LogSoftmax(dim=1)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout1(x)
                
        x = x.view(-1, 30 * 3 * 3) 
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        
        return self.softmax(x)

### 2nd Architecture

In [ ]:
'''
class ConvNN1(nn.Module):
    def __init__(self):
        super(ConvNN1, self).__init__()
        
        input_shape = 1,28,28
        Conv layer 1    :--> UNITS = 128 | KERNEL SIZE = 5 * 5 | STRIDE LENGTH = 1 | ACTIVATION = ReLu
        MaxPool layer 1 :--> MAX POOL WINDOW = 3 * 3 | STRIDE = 2
        model.add(Conv2D(128,kernel_size=(5,5), strides=1,padding='same',activation='relu',input_shape=(28,28,1)))
        model.add(MaxPool2D(pool_size=(3,3),strides=2,padding='same'))

        Conv layer 2    :--> UNITS = 64  | KERNEL SIZE = 3 * 3 | STRIDE LENGTH = 1 | ACTIVATION = ReLu
        MaxPool layer 2 :--> MAX POOL WINDOW = 2 * 2 | STRIDE = 2
        model.add(Conv2D(64,kernel_size=(2,2), strides=1,activation='relu',padding='same'))
        model.add(MaxPool2D((2,2),2,padding='same'))

        Conv layer 3    :--> UNITS = 32  | KERNEL SIZE = 2 * 2 | STRIDE LENGTH = 1 | ACTIVATION = ReLu
        MaxPool layer 3 :--> MAX POOL WINDOW = 2 * 2 | STRIDE = 2
        model.add(Conv2D(32,kernel_size=(2,2), strides=1,activation='relu',padding='same'))
        model.add(MaxPool2D((2,2),2,padding='same'))
                
        model.add(Flatten())
'''

### Initialize network

In [34]:
# Create the network and look at it's text representation
model = CNN().to(device)
#summary(model, (1, 28, 28))
print(model)

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=270, out_features=270, bias=True)
  (fc2): Linear(in_features=270, out_features=24, bias=True)
  (softmax): LogSoftmax(dim=1)
)


### Loss and optimizer

In [35]:
# loss_function
criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()

# optimizers require the parameters to optimize and a learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [48]:
print(torch.arange(0, 5))
print(torch.nn.functional.one_hot(torch.arange(0, 5), num_classes=5))

np

tensor([0, 1, 2, 3, 4])
tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]])


In [61]:
for batch_idx, (images, labels) in enumerate(x):
    print(np.array(labels))
    labels = torch.nn.functional.one_hot(labels.to(torch.int64), num_classes=24)
    print(labels.shape)
    outputs = model(images)
    print(outputs)
    break

[ 3.  2.  2. 11.  2. 11. 19. 13. 19. 12.  0.  6. 18. 11. 22.  5. 21.  0.
 20.  6. 17. 19. 14.  8.  0. 23.  3. 24. 15. 17. 12.  6. 22.  3. 17. 13.
  5. 23.  2. 24. 16. 16. 12. 22. 15.  7. 10. 18.  3.  0. 15. 21.  3.  1.
  6.  7. 18. 19. 15. 10. 12.  2.  0.  1.]
torch.Size([64, 25])
tensor([[-3.1756, -3.1643, -3.1601,  ..., -3.1113, -3.1904, -3.1904],
        [-3.1770, -3.1586, -3.1529,  ..., -3.1115, -3.1947, -3.1947],
        [-3.1950, -3.1794, -3.1351,  ..., -3.0959, -3.1993, -3.1993],
        ...,
        [-3.1871, -3.1677, -3.1621,  ..., -3.0936, -3.1969, -3.1969],
        [-3.1708, -3.1686, -3.1655,  ..., -3.0888, -3.1958, -3.1958],
        [-3.1972, -3.1914, -3.1791,  ..., -3.0748, -3.1972, -3.1972]],
       grad_fn=<LogSoftmaxBackward0>)


### Train the Model

In [37]:
n_total_steps = len(x)
        

for epoch in range(num_epochs): # no. of full passes (loop) over the data
    #running_loss = 0
    #print(f'epoch: {epoch+1}')

    for batch_idx, (images, labels) in enumerate(x):
        # get data as a list of [images, labels]
        # train_loader is a batch of featuresets and labels
        # batch_idx : index of the batch
        # images    : one batch of features
        # labels    : one batch of targets
        
        # get data to cuda if possible
        images = images.to(device=device)
        labels = labels.to(device=device)
        
        # images are in correct shape
        # no need to flatten MNIST images like normal neural network
        # we did it inside CNN class
        #images = print("images.shape:", images.shape)
        
        # forward propagation
        outputs = model(images) # (batch_size x num_classes)
        
        #output = net(X.view(-1, 28*28)) # pass in reshaped batch

        loss = criterion(outputs, labels)
        #loss = F.nll_loss(output, y)    # calc and grab loss value

        # zero previous gradients. you will do this likely every step
        optimizer.zero_grad()
        # back-propagation
        loss.backward()
        # gradient descent or adam step (optimize weights)
        optimizer.step()

        #running_loss += loss.item()


        if (batch_idx+1) % 100 == 0:
            print(f'epoch [{epoch+1}/{num_epochs}], step [{batch_idx+1}/{n_total_steps}], loss = {loss.item():.4f}')

    print("loss =", loss.item()) # print loss. we hope loss (a measure of wrong-ness) declines!
    print("==============================================================")


#print(f'Training loss: {running_loss / len(train_loader)}')

RuntimeError: expected scalar type Long but found Float

The parameters `kernel_size`, `stride`, `padding`, `dilation` can either be:
* a single `int` – in which case the same value is used for the height and width dimension
* a `tuple` of two ints – in which case, the first int is used for the height dimension, and the second int for the width dimension

In [ ]:
nn.Conv2d(
    in_channels, 
    out_channels, 
    kernel_size, 
    stride=1, 
    padding=0, 
    dilation=1, 
    groups=1, 
    bias=True, 
    padding_mode='zeros', 
    device=None, 
    dtype=None
    )